In [1]:
""" 제 2고지 자연스러운 코드로 """

' 제 2고지 자연스러운 코드로 '

In [2]:
""" STEP19. 변수 사용성 개선 """


import numpy as np

# ndarray 인스턴스만 취급하고록 바꿈 ( 다른게 들어오면 오류 )
class Variable:
    __array_priority = 200
    def __init__(self, data, name=None): # 생성자
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError(
                    '{}은(는) 지원하지 않습니다.'.format(type(data)))
         
        self.data = data # 변수의 데이터를 입력 데이터로 설정
        self.name = name # 변수 이름 저장
        self.grad = None # 미분값 저장
        self.creator = None # 연산을 나타내는 객체
        self.generation = 0 # 세대 수 기록

    # 해당 변수가 어떤 함수에 의해 만들어졌는지를 저장
    def set_creator(self, func):
        self.creator = func
        # 세대를 기록함 ( 부모 세대 + 1)
        self.generation = func.generation + 1

    # 역전파를 자동화 할 수 있도록 새로운 메서드 생성
    # retain_grad=False : 중간  변수의 미분값을모두 None으로 재설정
    def backward(self, retain_grad=False):
        # y.grad = np.array(1.0) 생략을 위한 if문
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop() # 함수를 가져온다.
            gys = [output().grad for output in f.outputs]
            gxs = f.backward(*gys) # 함수 f의 역전파 호출 ( 리스트 언팩 )
            
            # gxs가 튜플이 아니라면 튜플로 변환
            if not isinstance(gxs, tuple):
                gxs = (gxs, )

            # 역전파로 전파되는 미분값을 Variable인스턴스 변수 grad에 저장
            for x, gx in zip(f.inputs, gxs): # gxs와 f.inputs는 대응
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx
                    

                if x.creator is not None:
                    add_func(x.creator)

            # 중간 미분값 없앰
            if not retain_grad:
                for y in f.outputs:
                    y().grad = None # y는 약한 잠조 ( weakref )

    def cleargrad(self): # 미분값 초기화
        self.grad = None

    def __len__(self): # 객체 수
        return len(self.data)
    
    def __repr__(self): # 출력 설정
        if self.data is None:
            return 'variable(None)'
        p = str(self.data).replace('\n', '\n' + ' ' * 9)
        return 'variable(' + p + ')'
    
    """ STEP20. 연산자 오버로드 ( 1 ) """
    # def __mul__(self, other):
    #     return mul(self, other)

    @property
    def shape(self): # 다차원 배열의 형상
        return self.data.shape
    
    @property
    def ndim(self): # 차원 수
        return self.data.ndim
    
    @property
    def size(self): # 원소 수
        return self.data.size
    
    @property
    def dtype(self): # 데이터 타입
        return self.data.dtype


In [3]:
# 주어진 입력을 NumPy 배열로 변환하는 함수
def as_array(x):
    if np.isscalar(x):  # 입력이 스칼라인지 확인
        return np.array(x)  # 스칼라인 경우, 배열로 변환하여 반환
    return x  # 스칼라가 아닌 경우, 그대로 반환

# 역전파가 가능한지 여부
class Config:
    enable_backprop = True # True : 역전파 활성 모드

In [4]:
import contextlib

@contextlib.contextmanager
def using_config(name, value):
    # 설정값을 변경하기 전에 이전 설정값을 임시로 저장
    old_value = getattr(Config, name)
    # 지정된 속성에 새로운 값 설정
    setattr(Config, name, value)
    try:
        # 설정값을 변경한 후에 코드 블록을 실행
        yield
    finally:
        # 코드 블록 실행 후에 이전 설정값을 복원
        setattr(Config, name, old_value)

In [5]:
# with using_config('enable_backprop', False) 대신
def no_grad():
    return using_config('enable_backprop', False)

In [6]:
""" STEP21. 연산자 오버로드 ( 2 ) """


def as_variable(obj):
    if isinstance(obj, Variable):
        return obj
    return Variable(obj)

In [7]:
import weakref

class Function:
    # *ㅁㅁㅁ : 임의 개수의 인수 ( 가변길이 ) 를 건내 함수를 호출할 수 있음
    def __call__(self, *inputs):
        """ STEP21. 연산자 오버로드 ( 2 ) """
        inputs = [as_variable(x) for x in inputs]

        xs = [x.data for x in inputs]
        
        ys = self.forward(*xs) # 리스트 언팩 ( 원소를 낱개로 풀어서 전달 )

        if not isinstance(ys, tuple): # 튜플이 아닌 경우 추가 지원
            ys = (ys, )

        outputs = [Variable(as_array(y)) for y in ys]

        # enable_backprop = True 일 때만 역전파 코드 실행
        if Config.enable_backprop:
            # 역전파 시 노드에 따라 순서를 정하는데 사용
            self.generation = max([x.generation for x in inputs]) # 세대 설정

            # 각 output Variable 인스턴스의 creator를 현재 Function 객체로 설정
            for output in outputs:
                # 계산들의 연결을 만듬
                output.set_creator(self) # 연결 설정
            
            self.inputs = inputs # 순전파 결과값 기억
            self.outputs = [weakref.ref(output) for output in outputs]

        # 리스트의 원소가 하나라면 첫 번째 원소를 반환함
        return outputs if len(outputs) > 1 else outputs[0]
    
    # 순전파
    def forward(self, xs):
        raise NotImplementedError()
    
    # 역전파
    def backward(self, gys):
        raise NotImplementedError()

In [8]:
""" STEP22. 연산자 오버로드 ( 3 ) """


class Neg(Function):
    def forward(self, x):
        return -x
    
    def backward(self, gy):
        return -gy
    
def neg(x):
    return Neg()(x)

In [9]:
# 두 개의 입력을 받아 덧셈 수행
class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y
    
    def backward(self, gy):
        return gy, gy
    
def add(x0, x1):
    x1 = as_array(x1)
    return Add()(x0, x1)

In [10]:
class Sub(Function):
    def forward(self, x0, x1):
        y = x0 - x1
        return y
    
    def backward(self, gy):
        return gy, -gy
    
def sub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x0, x1)

def rsub(x0, x1):
    x1 = as_array(x1)
    return Sub()(x1, x0) # x0과 x1의 순서 바꿈

In [11]:
""" STEP20. 연산자 오버로드 ( 1 ) """


class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y
    
    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        # x0 미분값 = 출력값 * x1
        # x1 미분값 = 출력값 * x0
        return gy * x1, gy * x0
    
def mul(x0, x1):
    x0, x1 = as_array(x0), as_array(x1)
    return Mul()(x0, x1)

In [12]:
""" STEP22. 연산자 오버로드 ( 3 ) """


class Div(Function):
    def forward(self, x0, x1):
        y = x0 / x1
        return y
    
    def dackward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        gx0 = gy / x1
        gx1 = gy * (-x0 / x1 ** 2)
        return gx0, gx1
    
def div(x0, x1):
    x1 = as_array(x1)
    return Div()(x0, x1)

def rdiv(x0, x1):
    x1 = as_array(x1)
    return Div()(x1, x0) # x0과 x1의 순서 바꿈

In [13]:
class Pow(Function):
    def __init__(self, c):
        self.c = c # 상수

    def forward(self, x):
        y = x ** self.c
        return y
    
    def backward(self, gy):
        x = self.inputs[0].data
        c = self.c
        gx = c * x ** (c - 1) * gy
        return gx

def pow(x, c):
    return Pow(c)(x)

In [14]:
# y = x²
class Square(Function):
    # 순전파
    def forward(self, x):
        y = x ** 2 # y = x²
        return y
    
    # 역전파
    def backward(self, gy): # gy = 출력쪽에 전해지는 미분값을 전달하는 역할
        x = self.inputs[0].data # 수정전 : x = self.input.data
        gx = 2 * x * gy #  y' = 2x
        return gx

In [15]:
def square(x):
    return Square()(x)

In [16]:
# y = eˣ
class Exp(Function):
    # 순전파
    def forward(self, x):
        y = np.exp(x) # 주어진 입력값에 대한 지수 함수를 계산하여 반환
        return y
    
    # 역전파
    def backward(self, gy):
        x = self.input.data
        """ 지수 함수의 도함수는 자기 자신을 유지하므로 
            입력값의 지수 함수 값에 gy를 곱함 """
        gx = np.exp(x) * gy
        return gx

In [17]:
def exp(x):
    return Exp()(x)

In [18]:
Variable.__neg__ = neg
Variable.__add__ = add
Variable.__radd__ = add
Variable.__sub__ = sub
Variable.__rsub__ = rsub
Variable.__mul__ = mul
Variable.__rmul__ = mul
Variable.__div__ = div
Variable.__rdiv__ = rdiv
Variable.__pow__ = pow

In [19]:
x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
print(x.shape) # x.shap() 대신 x.shap으로 호출 가능

(2, 3)


In [20]:
print(len(x))

2


In [21]:
x = Variable(np.array([1, 2, 3]))
print(x)

x = Variable(None)
print(x)

x = Variable(np.array([[1, 2, 3], [4, 5, 6]]))
print(x)

variable([1 2 3])
variable(None)
variable([[1 2 3]
          [4 5 6]])


In [22]:
""" STEP20. 연산자 오버로드 ( 1 ) """


a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = add(mul(a, b), c)

y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
2.0
3.0


In [23]:
y = a * b
print(y)

variable(6.0)


In [24]:
y = a * b + c
y.backward()

print(y)
print(a.grad)
print(b.grad)

variable(7.0)
4.0
6.0


In [25]:
""" STEP21. 연산자 오버로드 ( 2 ) """


x = Variable(np.array(2.0))
y = x + np.array(3.0)
print(y)

variable(5.0)


In [26]:
y = x + 3.0
print(y)

variable(5.0)


In [27]:
y = 3.0 * x + 1.0

In [28]:
""" STEP22. 연산자 오버로드 ( 3 ) """


y = -x # 부호를 바꾼다
print(y)

variable(-2.0)


In [29]:
y = x ** 3
print(y)

variable(8.0)
